当你在保留99% 或者95% 或者其它百分比的方差时 结果表明 就只使用正则化将会给你 一种避免过拟合 绝对好的方法 ，同时正则化 效果也会比PCA更好 因为当你使用线性回归或者逻辑回归 或其他的方法 配合正则化时 这个最小化问题 实际就变成了y值是什么 才不至于 将有用的信息舍弃掉 然而PCA不需要使用到 这些标签  更容易将有价值信息舍弃 总之 使用PCA的目的是 加速 学习算法的时候是好的 但是用它来避免过拟合 却并不是一个好的PCA应用 我们使用正则化的方法来代替PCA方法 是很多人 建议的 。



你的学习算法 收敛地非常缓慢 占用内存 或者硬盘空间非常大 所以你想来压缩 数据 只有当你的x(i)效果不好 只有当你有证据或者 充足的理由来确定 x(i)效果不好的时候 那么就考虑用PCA来进行压缩数据 。


PCA通常都是 被用来 压缩数据的 以减少内存使用 或硬盘空间占用 或者用来可视化数据 
